In [ ]:
import numpy as np

from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField

from openff.system.utils import get_test_file_path
from openff.system.models import PotentialKey, TopologyKey
from openff.system.components.system import System

In [ ]:
# Load a minimal SMIRNOFF forcefield and Argon topology
argon_ff = ForceField(get_test_file_path("argon.offxml"))

mol = Molecule.from_smiles("[#18]")
mol.generate_conformers(n_conformers=1)

argon_top = Topology.from_molecules(10 * [mol])

In [ ]:
# Use a monkey-patched function to parametrize the topology against a force field
off_sys = System.from_smirnoff(force_field=argon_ff, topology=argon_top)

In [ ]:
# Look at which ParameterHandler objects from the OpenFF toolkit
# have been made into Potentialhandler objects
off_sys.handlers.keys()

In [ ]:
# Store this handler to inspect its contents
vdw = off_sys.handlers["vdW"]

In [ ]:
# Look at some ~metadata
vdw.type, vdw.expression, vdw.independent_variables

In [ ]:
# Return a mapping between atom indices and SMIRKS identifiers
vdw.slot_map

In [ ]:
# Return a mapping between SMIRKS identifiers and Potential objects;
# Note the de-duplication, resulting from a many-to-few mapping between
# atoms in the topology and unique parameters in the force field
vdw.potentials

In [ ]:
# Look at this contents of this Potential object
potential_key = PotentialKey(id="[#18:1]")
vdw.potentials[potential_key]

In [ ]:
# Further, look at the particular value of one of its parameters
vdw.potentials[potential_key].parameters["sigma"]

In [ ]:
# Look up, from the highest-level object, this same data, using the
# SMIRKS pattern as a key connecting the topological data to the
# parametrized data
topology_key = TopologyKey(atom_indices=(0,))
off_sys.handlers["vdW"].potentials[
    off_sys.handlers["vdW"].slot_map[topology_key]
].parameters["sigma"]